In [1]:
import os
import pickle
import gc

import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.regularizers import L1L2

from sklearn.model_selection import train_test_split

In [2]:
MODEL = "6-auto-regressive-lstm"
MAX_SAMPLES = 100_000

config = {
    "N_PREV": 120,
    "N_FORWARD": 48,
    "STEP_SIZE": 1,
    
    "LATENT_SIZE": 32,
    
    "LR": 0.001,
    "BATCH_SIZE": 256,
    "EPOCHS": 50,
    
    "L1": 0.0000001, 
    "L2": 0.0000001,

    "TARGET_COL": 'normalized_level',
}

In [3]:
import wandb
from wandb.keras import WandbCallback
from secrets import WANDB
wandb.login(key=WANDB)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\KiernanMcGuigan/.netrc


True

## **Load Data Function**

In [4]:
def get_samples(df, set_number):
    subset = df.loc[df.set==set_number, [config["TARGET_COL"]]]
    shape = subset.shape
    if(shape[0] > MAX_SAMPLES):
        subset = subset.iloc[-MAX_SAMPLES:, :]
    data = subset.values
    print(f'Data Shape: {shape}, Reduced Shape {data.shape}')
    X, y, dates = [], [], []
    for start_idx in range(0, data.shape[0]-config["N_PREV"]-config["N_FORWARD"], config["STEP_SIZE"]):
        mid_idx = start_idx + config["N_PREV"]
        end_idx = mid_idx + config["N_FORWARD"]
        X.append(data[start_idx:mid_idx])
        y.append(data[mid_idx:end_idx])
        dates.append(subset.iloc[mid_idx:end_idx].index.values)
    return np.array(X), np.array(y), np.array(dates)

def to_dataset(X, y):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    dataset = dataset.cache().shuffle(X.shape[0]+1).batch(config["BATCH_SIZE"]).prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

def get_train_and_val_datasets(X, y):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, shuffle=True)
    train_ds = to_dataset(X_train, y_train)
    val_ds = to_dataset(X_val, y_val)
    return train_ds, val_ds

def df_to_samples(df):
    X, y, _ = get_samples(df, 0)
    train_ds, val_ds = get_train_and_val_datasets(X, y)
    X_test, y_test, dates = get_samples(df, 1)
    return train_ds, val_ds, X_test, y_test, dates
    
def get_data(fold):
    df = pd.read_csv(f'data/fold{fold}_data_v3.csv')
    with open(f'./data/fold{fold}_normalizer_v3.pkl', mode='rb') as f:
        normalizer = pickle.load(f)
    return df_to_samples(df), normalizer

## **Model Functions**

In [5]:
from tensorflow.keras import layers
class MyModel(tf.keras.models.Model):
    def __init__(self, inputs=config['N_PREV'], outputs=config['N_FORWARD'], latent_size=config['LATENT_SIZE'], l1=config['L1'], l2=config['L2']):
        super().__init__()
        self.inputs = inputs
        self.outputs = outputs
        self.latent_size = latent_size
        self.l1 = l1
        self.l2 = l2
        self.lstm1 = layers.LSTM(self.latent_size, return_state=True, return_sequences=True, recurrent_initializer='glorot_uniform', activity_regularizer=L1L2(l1=self.l1,l2=self.l2))
        self.lstm2 = layers.LSTM(self.latent_size, return_state=True, return_sequences=True, recurrent_initializer='glorot_uniform', activity_regularizer=L1L2(l1=self.l1,l2=self.l2))
        self.lstm3 = layers.LSTM(self.latent_size, return_state=True, return_sequences=False, recurrent_initializer='glorot_uniform', activity_regularizer=L1L2(l1=self.l1,l2=self.l2))
        self.dense = layers.Dense(1)
        self.concat = layers.Lambda(lambda x: K.concatenate(x, axis=1))
        
    def get_config(self):
        config = super().get_config()
        config.update({
            'inputs':self.inputs,
            'outputs':self.outputs,
            'latent_size':self.latent_size,
            'l1':self.l1,
            'l2':self.l2,
        })
        return config
    
    @tf.function
    def call(self, inputs, training=None):
        predictions = []
#         print('Inputs shape: ', inputs.shape)
        x, hidden_state1, cell_state1 = self.lstm1(inputs, training=training)
#         print('lstm1 shape: ', x.shape)
        x, hidden_state2, cell_state2 = self.lstm2(x, training=training)
#         print('lstm2 shape: ', x.shape)
        x, hidden_state3, cell_state3 = self.lstm3(x, training=training)
#         print('lstm3 shape: ', x.shape)
        
        prediction = self.dense(x)
#         print('prediction shape: ', prediction.shape)
        predictions.append(prediction)
        
        for _ in range(1, self.outputs):
            x = tf.expand_dims(prediction, axis=1)
#             print('expanded x shape: ', x.shape)
            x, hidden_state1, cell_state1 = self.lstm1(x, initial_state=[hidden_state1,cell_state1], training=training)
#             print('lstm1 shape: ', x.shape)
            x, hidden_state2, cell_state2 = self.lstm2(x, initial_state=[hidden_state2,cell_state2], training=training)
#             print('lstm2 shape: ', x.shape)
            x, hidden_state3, cell_state3 = self.lstm3(x, initial_state=[hidden_state3,cell_state3], training=training)
#             print('lstm3 shape: ', x.shape)
            prediction = self.dense(x)
#             print('prediction shape: ', prediction.shape)
            predictions.append(prediction)
        
        predictions = self.concat(predictions)
#         print('predictions shape: ', prediction.shape)
        return predictions

In [6]:
def get_model():
    inputs = tf.keras.layers.Input(shape=(config["N_PREV"], 1))
    lstm1 = tf.keras.layers.LSTM(config["LATENT_SIZE"], 
                                 return_sequences=True, 
                                 return_state=True, 
                                 recurrent_initializer='glorot_uniform', 
                                 activity_regularizer=L1L2(l1=config['L1'],l2=config['L2']))
    lstm2 = tf.keras.layers.LSTM(config["LATENT_SIZE"], 
                                 return_sequences=True, 
                                 return_state=True, 
                                 recurrent_initializer='glorot_uniform', 
                                 activity_regularizer=L1L2(l1=config['L1'],l2=config['L2']))
    lstm3 = tf.keras.layers.LSTM(config["LATENT_SIZE"], 
                                 return_sequences=False, 
                                 return_state=True, 
                                 recurrent_initializer='glorot_uniform', 
                                 activity_regularizer=L1L2(l1=config['L1'],l2=config['L2']))
    dense = tf.keras.layers.Dense(1)
    
    predictions = []
    x, hidden_state1, cell_state1 = lstm1(inputs)
    x, hidden_state2, cell_state2 = lstm2(x)
    x, hidden_state3, cell_state3 = lstm3(x)
    x = dense(x)
    
    predictions.append(x)
    for _ in range(1, config['N_FORWARD']):
        x = tf.expand_dims(x, axis=1)
        x, hidden_state1, cell_state1 = lstm1(x, initial_state=[hidden_state1, cell_state1])
        x, hidden_state2, cell_state2 = lstm2(x, initial_state=[hidden_state2, cell_state2])
        x, hidden_state3, cell_state3 = lstm3(x, initial_state=[hidden_state3, cell_state3])
        x = dense(x)
        
        predictions.append(x)
        
    outputs = tf.keras.layers.Lambda(lambda x: K.concatenate(x, axis=1))(predictions)
    model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
    model.compile(loss="mse", metrics=["mae"], optimizer=tf.keras.optimizers.Adam(learning_rate=config["LR"]))
    return model

# def get_model():
#     model = MyModel()
#     model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(config["LR"]), metrics=['mae'])
#     model.predict(tf.ones(shape=(config['BATCH_SIZE'],config['N_PREV'],1)))
#     return model
    
test_model = get_model()
print(test_model.summary())
del test_model
gc.collect()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 120, 1)]     0           []                               
                                                                                                  
 lstm (LSTM)                    multiple             4352        ['input_1[0][0]',                
                                                                  'tf.expand_dims[0][0]',         
                                                                  'lstm[0][1]',                   
                                                                  'lstm[0][2]',                   
                                                                  'tf.expand_dims_1[0][0]',       
                                                                  'lstm[1][1]',               

                                                                  'lstm[25][2]',                  
                                                                  'tf.expand_dims_26[0][0]',      
                                                                  'lstm[26][1]',                  
                                                                  'lstm[26][2]',                  
                                                                  'tf.expand_dims_27[0][0]',      
                                                                  'lstm[27][1]',                  
                                                                  'lstm[27][2]',                  
                                                                  'tf.expand_dims_28[0][0]',      
                                                                  'lstm[28][1]',                  
                                                                  'lstm[28][2]',                  
          

                                                                  'lstm_1[5][2]',                 
                                                                  'lstm[7][0]',                   
                                                                  'lstm_1[6][1]',                 
                                                                  'lstm_1[6][2]',                 
                                                                  'lstm[8][0]',                   
                                                                  'lstm_1[7][1]',                 
                                                                  'lstm_1[7][2]',                 
                                                                  'lstm[9][0]',                   
                                                                  'lstm_1[8][1]',                 
                                                                  'lstm_1[8][2]',                 
          

                                                                  'lstm_1[33][1]',                
                                                                  'lstm_1[33][2]',                
                                                                  'lstm[35][0]',                  
                                                                  'lstm_1[34][1]',                
                                                                  'lstm_1[34][2]',                
                                                                  'lstm[36][0]',                  
                                                                  'lstm_1[35][1]',                
                                                                  'lstm_1[35][2]',                
                                                                  'lstm[37][0]',                  
                                                                  'lstm_1[36][1]',                
          

                                                                  'lstm_2[13][1]',                
                                                                  'lstm_2[13][2]',                
                                                                  'lstm_1[15][0]',                
                                                                  'lstm_2[14][1]',                
                                                                  'lstm_2[14][2]',                
                                                                  'lstm_1[16][0]',                
                                                                  'lstm_2[15][1]',                
                                                                  'lstm_2[15][2]',                
                                                                  'lstm_1[17][0]',                
                                                                  'lstm_2[16][1]',                
          

                                                                  'lstm_1[42][0]',                
                                                                  'lstm_2[41][1]',                
                                                                  'lstm_2[41][2]',                
                                                                  'lstm_1[43][0]',                
                                                                  'lstm_2[42][1]',                
                                                                  'lstm_2[42][2]',                
                                                                  'lstm_1[44][0]',                
                                                                  'lstm_2[43][1]',                
                                                                  'lstm_2[43][2]',                
                                                                  'lstm_1[45][0]',                
          

                                                                                                  
 tf.expand_dims_8 (TFOpLambda)  (None, 1, 1)         0           ['dense[8][0]']                  
                                                                                                  
 tf.expand_dims_9 (TFOpLambda)  (None, 1, 1)         0           ['dense[9][0]']                  
                                                                                                  
 tf.expand_dims_10 (TFOpLambda)  (None, 1, 1)        0           ['dense[10][0]']                 
                                                                                                  
 tf.expand_dims_11 (TFOpLambda)  (None, 1, 1)        0           ['dense[11][0]']                 
                                                                                                  
 tf.expand_dims_12 (TFOpLambda)  (None, 1, 1)        0           ['dense[12][0]']                 
          

                                                                  'dense[4][0]',                  
                                                                  'dense[5][0]',                  
                                                                  'dense[6][0]',                  
                                                                  'dense[7][0]',                  
                                                                  'dense[8][0]',                  
                                                                  'dense[9][0]',                  
                                                                  'dense[10][0]',                 
                                                                  'dense[11][0]',                 
                                                                  'dense[12][0]',                 
                                                                  'dense[13][0]',                 
          

508453

## **Train Loop**

In [7]:
def train_model(model, train_ds, val_ds):
    reducer = tf.keras.callbacks.ReduceLROnPlateau(monior='val_loss', factor=0.1, patience=2, mode='min', cooldown=1)
    stopper = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4, mode='min', restore_best_weights=True)
    model.fit(train_ds,
              epochs=config["EPOCHS"], 
              callbacks=[reducer, stopper, WandbCallback()],
              validation_data=val_ds)
    return model

def test_model(model, X_test, y_test, normalizer):
    predictions = model.predict(X_test)
    inversed_predictions, inversed_actuals = [], []
    for pred, actual in zip(predictions, y_test):
        inversed_predictions.append(normalizer.inverse_transform(np.array(pred).reshape(-1,1)).reshape((-1)))
        inversed_actuals.append(normalizer.inverse_transform(np.array(actual).reshape(-1,1)).reshape((-1)))
    inversed_predictions = np.array(inversed_predictions)
    inversed_actuals = np.array(inversed_actuals)
    forward_error = np.mean(np.abs(inversed_actuals - inversed_predictions), axis=-1)
    error = np.mean(forward_error)
    return error, forward_error, inversed_predictions, inversed_actuals

def record_forward_error(forward_errors, error, run):
    run.log({'test/error': error})
    run.log({'test/forward_error': wandb.plot.line_series(
        xs=[i+1 for i in range(forward_errors.shape[0])],
        ys=[forward_errors],
        keys=['Forward Horizon Errors'],
        title=f'Forward Horizon Errors (MAE: {error})',
    )})
        
def graph_predictions(predictions, actuals, dates, run):
    print(predictions.shape, actuals.shape, dates.shape)
    for look_ahead in [1, 12, 24, 36, 48]:
        preds, true, time = predictions[:, look_ahead-1], actuals[:, look_ahead-1], dates[:, look_ahead-1]
        error = np.mean(np.abs(true - preds))
        run.log({f'test/forecast-{look_ahead}-hours-ahead': wandb.plot.line_series(
            xs=time,
            ys=[true, preds],
            keys=['Actuals', 'Predictions'],
            title=f'{look_ahead} Hour(s) Ahead Forecast (MAE: {error})',
            xname='week'
        )})

In [8]:
def train():
    for fold in range(1, 4):
        run = wandb.init(project="time-series-methods", entity="kmcguigan", group=f"{MODEL}-model", config=config, job_type="train")
        run.name = f'{MODEL}-fold-{fold}'
        (train_ds, val_ds, X_test, y_test, dates), normalizer = get_data(fold)
        model = get_model()
        model = train_model(model, train_ds, val_ds)
        error, forward_errors, predictions, actuals = test_model(model, X_test, y_test, normalizer)
        run.log({'test-error': error})
        record_forward_error(forward_errors, error, run)
        graph_predictions(predictions, actuals, dates, run)
        run.finish()
        del model
        del train_ds
        del val_ds
        gc.collect()
    return

In [ ]:
train()

wandb: Currently logged in as: kmcguigan. Use `wandb login --relogin` to force relogin


Data Shape: (159096, 1), Reduced Shape (100000, 1)
Data Shape: (43390, 1), Reduced Shape (43390, 1)


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/50
235/332 [====================>.........] - ETA: 49s - loss: 0.4797 - mae: 0.5325